In [1]:
from pyniryo import *

In [2]:
robot_ip_address = "169.254.200.200"

In [3]:
robot_ip_address = "10.10.10.10"

In [4]:
robot = NiryoRobot(robot_ip_address)


Connected to server (10.10.10.10) on port: 40001



In [5]:
robot.calibrate_auto()

In [24]:
base_model = PoseObject(
x=0.5, y=0, z=0.25,
roll=-0, pitch=0, yaw=0
)
robot.move_pose(base_model)

In [ ]:
robot.get_joints()

[-0.0007288597570993538,
 -1.2958018047396962,
 1.2672217058481852,
 -0.0014413271980928677,
 0.029052981380034204,
 0.0016266343776782932]

In [25]:
robot.move_joints(0, 0, 0, 0, 0, 0)

In [23]:
robot.get_pose()

x = 0.1401, y = -0.0001, z = 0.2031
roll = 0.002, pitch = 0.754, yaw = -0.001

In [ ]:
robot.update_tool()

In [22]:
robot.close_gripper(speed=500)

In [20]:
robot.open_gripper(speed=500)

In [17]:
pick_pose = PoseObject(
x=0.1, y=0.2, z=0.4,
roll=0, pitch=0, yaw=0
)

robot.move_pose(pick_pose)
# robot.move_pose(pick_pose)


In [21]:
joints_read = robot.get_joints()
joints_read

[-0.0007288597570993538,
 0.49940895727663126,
 -1.2506181983468665,
 9.265358979293481e-05,
 -0.0031606151655640957,
 0.0016266343776782932]

In [26]:
robot.close_connection()


Disconnected from robot

